## First, imports:

In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units

Import my library:

In [ ]:
import os
import sys

apt_path = os.path.abspath(os.path.join('..', 'apostletools'))
sys.path.append(apt_path)

import snapshot

In [ ]:
import importlib
importlib.reload(snapshot)

# Radial Distribution of Different Mass Components of the M31 and the MW


## Get data

In [ ]:
snap_id = 127
data = {"plain-LCDM" : 
        {"M31" : {"halo_id" : (1,0)},
         "MW" : {"halo_id" : (2,0)}
        },
        "curv-p082" : 
        {"M31" : {"halo_id" : (1,0)},
         "MW" : {"halo_id" : (1,1)}
        }
       }

linestyle = {'all' : ':', 'gas' : '-', 'dm' : '--', 
             'stars' : '-', 'bh' : '--'}

for colors, sim_name in zip([("black", "gray"), ("red", "pink")],
                            data.keys()):
    
    color = {'all' : colors[1], 'gas' : colors[1], 'dm' : colors[0], 
             'stars' : colors[0], 'bh' : colors[1]}

    for galaxy in data[sim_name].keys():
        data[sim_name][galaxy].update(
        {"Color" : color, "Linestyle" : linestyle})

In [ ]:
for sim_id, sim_name in zip(["V1_MR_fix", "V1_MR_curvaton_p082_fix"], data.keys()):
    snap = snapshot.Snapshot(sim_id=sim_id, snap_id=snap_id)
        
    for galaxy in data[sim_name].keys():
        idx = snap.index_of_halo(data[sim_name][galaxy]["halo_id"][0], 
                                 data[sim_name][galaxy]["halo_id"][1])
        v_circ = {}
        radii = {}
        for pt, pt_id in zip(['all', 'gas', 'dm', 'stars', 'bh'],
                             ['All', 'PartType0', 'PartType1', 'PartType4', 'PartType5']):
            rot_curves = snap.get_subhalos(
                'Vcirc', h5_group='Extended/RotationCurve/{}'.format(pt_id))
            sub_offset = snap.get_subhalos(
                'SubOffset', h5_group='Extended/RotationCurve/{}'.format(pt_id))
            
            v_circ[pt] = rot_curves[sub_offset[idx]:sub_offset[idx+1],0] \
                * units.cm.to(units.km)
            radii[pt] = rot_curves[sub_offset[idx]:sub_offset[idx+1],1] \
                * units.cm.to(units.kpc)

        data[sim_name][galaxy]["Vcirc"] = v_circ
        data[sim_name][galaxy]["Radius"] = radii

In [ ]:
# Construct saving location:
filename = "rotation_curves_of_centrals_by_type.png"
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

In [ ]:
# Choose font sizes:
parameters = {'axes.titlesize': 10,
              'axes.labelsize': 8,
              'xtick.labelsize': 7,
              'ytick.labelsize': 7,
              'legend.fontsize': 8,
              'legend.title_fontsize': 7}

In [ ]:
# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

fig, axes = plt.subplots(ncols=2, sharey="row", figsize=(6,2.8))
plt.subplots_adjust(wspace=0)

# Set titles:
axes[0].set_title("Andromeda")
axes[1].set_title("Milky Way")

# Set axes:
x_down = 0; x_up = 59
y_down = 0; y_up = 270
axes[0].set_xlim(x_down, x_up)
axes[1].set_xlim(x_down, x_up)
axes[0].set_ylim(y_down,y_up)

# Set labels:
axes[0].set_xlabel('$r[\mathrm{kpc}]$')
axes[1].set_xlabel('$r[\mathrm{kpc}]$')
axes[0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$')

# Galaxy text:
#axes.set_title('Rotation curve: GN={}, SGN={}'.format(gn,sgn))

for i, sim_name in enumerate(data.keys()):
    
    # Plot M31:
    for v_circ, r, col, ls in zip(
        data[sim_name]["M31"]["Vcirc"].values(),
        data[sim_name]["M31"]["Radius"].values(),
        data[sim_name]["M31"]["Color"].values(),
        data[sim_name]["M31"]["Linestyle"].values()):

        axes[0].plot(r, v_circ, c=col, linestyle=ls)
    
    # Plot MW:
    for v_circ, r, col, ls in zip(
        data[sim_name]["MW"]["Vcirc"].values(),
        data[sim_name]["MW"]["Radius"].values(),
        data[sim_name]["MW"]["Color"].values(),
        data[sim_name]["MW"]["Linestyle"].values()):

        axes[1].plot(r, v_circ, c=col, linestyle=ls)
         
            
# Make dummy plots for the legend:   
l1, = axes[1].plot([], [], c=data["plain-LCDM"]["M31"]["Color"]["stars"])
l2, = axes[1].plot([], [], c=data["curv-p082"]["M31"]["Color"]["stars"])
sim_legend = axes[1].legend([l1,l2], ["plain-LCDM", "curv-p082"],
                            loc="upper left")
axes[1].add_artist(sim_legend)

for pt in data[sim_name]["MW"]["Color"].keys():
    axes[1].plot([], [], 
                 c=data["plain-LCDM"]["MW"]["Color"][pt],
                 linestyle=data["plain-LCDM"]["MW"]["Linestyle"][pt],
                 label=pt)
    
axes[1].legend(loc="upper right")
        
    
plt.legend(loc="upper right")
plt.tight_layout()

plt.savefig(os.path.join(path,filename), dpi=300, bbox_inches='tight')